Use CUDA enabled whisper.cpp. Setup is outlined in https://github.com/ggerganov/whisper.cpp. YouTube extraction code from OPENVINO example notebook 227-whisper-nnfc-quantize.ipynb.

Enter youtube playlist link. Downloads videos is lowest resolution, transcribes them, combines timestamped transcription into an html file for Send to Kindle. Other transcriptions are also provided such as txt, substitle text, etc.

In [9]:
import numpy as np
import subprocess
import os
import glob
import subprocess
import ipywidgets as widgets
from pytube import YouTube
from utils import get_audio
from utils import prepare_srt
from pathlib import Path
from pytube import Playlist
import datetime

np.set_printoptions(linewidth=50)

# change to reflect your local directory and file name
home_directory = os.path.expanduser("~")
# create a separate directory for youtube videos
directory = home_directory + '/machine_learning/whisper.cpp/youtube/'

In [2]:
# playlist conversion to yourtube video links from deepseeker ai
def get_video_links(playlist_url):
    playlist = Playlist(playlist_url)
    video_links = []
    for video_url in playlist.video_urls:
        video_links.append(video_url)
    return video_links

# ENTER youtube playlist here:
playlist_url = 'https://youtube.com/playlist?list=PLYXvCE1En13epbogBmgafC_Yyyk9oQogl&feature=shared'  # replace with your playlist URL
video_links = get_video_links(playlist_url)
for link in video_links:
    print(link)

https://www.youtube.com/watch?v=fjbTCPAj254
https://www.youtube.com/watch?v=wR81RH-GI9U
https://www.youtube.com/watch?v=VU73LRk8Zjw
https://www.youtube.com/watch?v=lCCT9JGkLw8
https://www.youtube.com/watch?v=739Ddvx1OIU
https://www.youtube.com/watch?v=72QWY6ubS14
https://www.youtube.com/watch?v=Szae6pHKr60
https://www.youtube.com/watch?v=_Hxt8CibzIs
https://www.youtube.com/watch?v=n3WxDCvjlAs
https://www.youtube.com/watch?v=YEOAn9FIvyI
https://www.youtube.com/watch?v=QwTBtKWmA0M


In [8]:
# bulk transcription of each youtube video:
for link in video_links:
    VIDEO_LINK = link
    # name after linkID
    name = link[-11:]
    link = widgets.Text(
        value=VIDEO_LINK,
        placeholder="Type link for video",
        description="Video:",
        disabled=False
    )
    link
    print(f"Downloading video {link.value} started")
    print(name)
    output_file = Path(directory + name + ".mp4")
    yt = YouTube(link.value)
    # set to .get_highest_resolution if you want to download highest resolution video
    yt.streams.get_lowest_resolution().download(filename=output_file)
    print(f"Video saved to {output_file}")

    # audio must always be 16hz 16 format .wav:
    extracted_audio_file = name + '.wav'
    def extract_audio(video_path, audio_path):
        yes_command = f'echo "y" | '
        command = yes_command + "ffmpeg -i {} -vn -acodec pcm_s16le -ar 16000 -ac 1 {}".format(video_path, audio_path)
        subprocess.call(command, shell=True)
    try:
        extract_audio(output_file, directory + extracted_audio_file)
        print("Audio coverted successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Audio convertion failed with error {e.returncode}.")

    # transcribe using the base model (great with CUDA enabled whisper.cpp)
    try:
        subprocess.run(['transcribe -t 12 -m ' + home_directory + '/machine_learning/whisper.cpp/models/ggml-base.en.bin -f ' 
                    + directory + extracted_audio_file + ' -otxt -ovtt -osrt -olrc'], shell=True, check=True)
        print("Transcription executed successfully and saved in " + directory)
    except subprocess.CalledProcessError as e:
        print(f"Transcription failed with error {e.returncode}.")

fjbTCPAj254
Video saved to /var/home/fraser/machine_learning/whisper.cpp/samples/fjbTCPAj254.mp4


ffmpeg version 4.4 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 9.3.0 (crosstool-NG 1.24.0.133_b0863d8_dirty)
  configuration: --prefix=/root/miniconda3/envs/conda_bld/conda-bld/ffmpeg_1635335682798/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place --cc=/root/miniconda3/envs/conda_bld/conda-bld/ffmpeg_1635335682798/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 70.100 / 56. 70.100
  libavcodec     58.134.100 / 58.134.100
  libavformat    58. 76.100 / 58. 76.100
  libavdevice    58. 13.100 / 58. 13.100
  libavfilter     7.110.100 /  7.110.100
  libavresample   4.  0.  0 /  4.  0.  0

Audio coverted successfully.


whisper_init_from_file_with_params_no_state: loading model from '/var/home/fraser/machine_learning/whisper.cpp/models/ggml-base.en.bin'
whisper_model_load: loading model
whisper_model_load: n_vocab       = 51864
whisper_model_load: n_audio_ctx   = 1500
whisper_model_load: n_audio_state = 512
whisper_model_load: n_audio_head  = 8
whisper_model_load: n_audio_layer = 6
whisper_model_load: n_text_ctx    = 448
whisper_model_load: n_text_state  = 512
whisper_model_load: n_text_head   = 8
whisper_model_load: n_text_layer  = 6
whisper_model_load: n_mels        = 80
whisper_model_load: ftype         = 1
whisper_model_load: qntvr         = 0
whisper_model_load: type          = 2 (base)
whisper_model_load: adding 1607 extra tokens
whisper_model_load: n_langs       = 99
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA RTX A1000 Laptop GPU, compute capability 8.6, VMM: yes
whisper_backend_init: using 


[00:00:00.000 --> 00:00:12.980]   So, maybe let's start. So, today we will continue talking about the lower bound for reinforcement
[00:00:12.980 --> 00:00:18.420]   learning. So, in the last lecture, let's just first recap a bit what are we having in talking
[00:00:18.420 --> 00:00:22.980]   about in the last lecture. In the last lecture, we talk about the general strategy to develop
[00:00:22.980 --> 00:00:34.040]   a lower bound is we can perform this reduction, where we will reduce the algorithm, the lower
[00:00:34.040 --> 00:00:46.640]   bounds for any randomized algorithm. And we need to construct a hard instance which is
[00:00:46.640 --> 00:00:57.700]   specific to this algorithm. And the expected regret, this is the original target of lower
[00:00:57.700 --> 00:01:05.960]   bounds, which trying to prove this is like small. This is lower bounded by something. And
[00:01:05.960 --> 00:01:11.880]   we say by reduction, we can actually reduce it by some lower bound, which is to 

...

ffmpeg version 4.4 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 9.3.0 (crosstool-NG 1.24.0.133_b0863d8_dirty)
  configuration: --prefix=/root/miniconda3/envs/conda_bld/conda-bld/ffmpeg_1635335682798/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place --cc=/root/miniconda3/envs/conda_bld/conda-bld/ffmpeg_1635335682798/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 70.100 / 56. 70.100
  libavcodec     58.134.100 / 58.134.100
  libavformat    58. 76.100 / 58. 76.100
  libavdevice    58. 13.100 / 58. 13.100
  libavfilter     7.110.100 /  7.110.100
  libavresample   4.  0.  0 /  4.  0.  0

Audio coverted successfully.


whisper_init_from_file_with_params_no_state: loading model from '/var/home/fraser/machine_learning/whisper.cpp/models/ggml-base.en.bin'
whisper_model_load: loading model
whisper_model_load: n_vocab       = 51864
whisper_model_load: n_audio_ctx   = 1500
whisper_model_load: n_audio_state = 512
whisper_model_load: n_audio_head  = 8
whisper_model_load: n_audio_layer = 6
whisper_model_load: n_text_ctx    = 448
whisper_model_load: n_text_state  = 512
whisper_model_load: n_text_head   = 8
whisper_model_load: n_text_layer  = 6
whisper_model_load: n_mels        = 80
whisper_model_load: ftype         = 1
whisper_model_load: qntvr         = 0
whisper_model_load: type          = 2 (base)
whisper_model_load: adding 1607 extra tokens
whisper_model_load: n_langs       = 99
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA RTX A1000 Laptop GPU, compute capability 8.6, VMM: yes
whisper_backend_init: using 


[00:00:00.000 --> 00:00:03.120]   [MUSIC]
[00:00:03.120 --> 00:00:05.760]   >> Hello everyone, I'm Chijing,
[00:00:05.760 --> 00:00:07.960]   I mean from ECE departments.
[00:00:07.960 --> 00:00:11.240]   So this year we will be working together on
[00:00:11.240 --> 00:00:14.560]   this course called Foundations of Reinforcement Learning.
[00:00:14.560 --> 00:00:16.480]   It's called Foundations but it's really
[00:00:16.480 --> 00:00:19.760]   the mathematical foundations of Reinforcement Learning.
[00:00:19.760 --> 00:00:23.760]   As you can see and this course will be videotaped,
[00:00:23.760 --> 00:00:27.320]   but it's basically for like open lecture and
[00:00:27.320 --> 00:00:31.160]   for external students and you are still very
[00:00:31.160 --> 00:00:34.000]   welcome to ask questions if you have anything you're unclear,
[00:00:34.000 --> 00:00:37.600]   like feel free to ask and they will be all very valuable.
[00:00:37.600 --> 00:00:41.880]   We'll first do a very brief i

output_txt: saving output to '/var/home/fraser/machine_learning/whisper.cpp/samples/QwTBtKWmA0M.wav.txt'
output_vtt: saving output to '/var/home/fraser/machine_learning/whisper.cpp/samples/QwTBtKWmA0M.wav.vtt'
output_srt: saving output to '/var/home/fraser/machine_learning/whisper.cpp/samples/QwTBtKWmA0M.wav.srt'
output_lrc: saving output to '/var/home/fraser/machine_learning/whisper.cpp/samples/QwTBtKWmA0M.wav.lrc'

whisper_print_timings:     load time =  1284.46 ms
whisper_print_timings:     fallbacks =   4 p /   3 h
whisper_print_timings:      mel time =  2790.22 ms
whisper_print_timings:   sample time = 27485.85 ms / 69271 runs (    0.40 ms per run)
whisper_print_timings:   encode time =   362.51 ms /   199 runs (    1.82 ms per run)
whisper_print_timings:   decode time =   982.25 ms /   514 runs (    1.91 ms per run)
whisper_print_timings:   batchd time = 35455.22 ms / 67752 runs (    0.52 ms per run)
whisper_print_timings:   prompt time = 10019.92 ms / 44203 runs (    0.23 ms per

20 minutes to download and transcribe nearly 14 hours of video

In [13]:
# helper utility to combine all transcripts into one file 
# code assumes you want youtube list sorted reverse chronological order so the oldest video is first
# comment out markdown_files.sort() if you want it the other way (newest video first)
# most courses will have lesson 1 as the oldest so the default should be fine
# ai assisted code modified

def combine_markdown_files(output_file_path, input_directory):
    # .lrc files have time stamps plus text in easy to read format
    markdown_files = glob.glob(input_directory + '*.lrc')

    # sort files in reverse timestamp order for youtube lists
    # that have latest video first
    markdown_files.sort(key=os.path.getmtime, reverse=True)

    with open(output_file_path, 'w') as output_file:
        for file_path in markdown_files:
            file_name = os.path.basename(file_path)
            with open(file_path, 'r') as input_file:
                output_file.write(f"## {file_name}\n\n")
                output_file.write(input_file.read())
                output_file.write('\n\n')  # Add newline between files

    print(f"Combined {len(markdown_files)} text files into {output_file_path}")

# saves combined text files into directory of Jupyter notebook (not the input directory)
# Get today's date in the format YYYYMMDD
date_string = datetime.datetime.now().strftime("%Y%m%d")
filename = "youtube-transcription"
# Append date to filename
filename_with_date = f"{filename}_{date_string}.txt"
combine_markdown_files(filename_with_date, directory)

Combined 11 text files into youtube-transcription_20240320.txt


In [14]:
# need to convert text file to html because send to kindle joins lines in txt file
def convert_txt_to_html(txt_file_path, html_file_path):
    with open(txt_file_path, 'r') as txt_file, open(html_file_path, 'w') as html_file:
        html_file.write('<!DOCTYPE html>\n<html>\n<body>\n')
        for line in txt_file:
            html_file.write('<p>{}</p>\n'.format(line.strip()))
        html_file.write('</body>\n</html>\n')

# convert to html; note file extensions
txt_file_path = filename_with_date                      # replace with your text file path
html_file_path = filename_with_date + '.html'           # replace with your HTML file path
convert_txt_to_html(txt_file_path, html_file_path)